In [5]:
import sys, os
import pandas as pd

sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")

from Clust.clust.ML.common.common import p1_integratedDataSaving as p1
import pathSetting
import matplotlib.pyplot as plt 

# 1. Set datasetName and related parameters

In [ ]:
### 1-0. Set DataSetNameList
datasetNameList=['Hs1SwineFarmWithWeatherTime', 'gunwiStrawberryWithWeatherTime', 'strawberryOpenTime']

### 1-1. Change Variables#########################################################
datasetName = datasetNameList[0]
DataSaveMode='CSV' #CSV or influx
integration_freq_sec = 60 * 5 # 5분
#################################################################################

In [ ]:
## 1-2. PreDefine Condition
# if datasetName=='Hs1SwineFarmWithWeatherTime':
#     dataInfo = [['farm_swine_air', 'Sadle'], ['weather_outdoor_keti_clean', 'sangju'], ['life_additional_Info', 'trigonometicInfoByHours']]
#     trainStartTime = "2020-11-25 00:00:00"
#     trainEndTime ="2020-12-25 00:00:00"
#     testStartTime ="2020-12-26 00:00:00"
#     testEndTime ="2021-01-03 00:00:00"
    
    
if datasetName=='Hs1SwineFarmWithWeatherTime':
    trainStartTime = "2021-02-01 00:00:00"
    trainEndTime ="2021-03-10 00:00:00"
    testStartTime ="2021-03-10 00:00:00"
    testEndTime ="2021-03-17 00:00:00"
    dataInfo = [['farm_swine_air', 'HS2'], ['weather_outdoor_keti_clean', 'sangju'], ['life_additional_Info', 'trigonometicInfoByHours']]
    
elif datasetName=='gunwiStrawberryWithWeatherTime':
    dataInfo = [['farm_strawberry_gunwi', 'control_environment'], ['farm_strawberry_gunwi', 'environment'], ['life_additional_Info', 'trigonometicInfoByHours']]
    trainStartTime = "2022-01-22 00:00:00"
    trainEndTime ="2022-02-25 00:00:00"
    testStartTime ="2022-02-25 00:00:00"
    testEndTime ="2022-02-28 00:00:00"
    
elif datasetName =='strawberryOpenTime':
    dataInfo = [['farm_strawberry_gunwi', 'control_environment'], ['farm_strawberry_gunwi', 'environment'], ['life_additional_Info', 'trigonometicInfoByHours']]
    trainStartTime = "2022-01-22 00:00:00"
    trainEndTime ="2022-02-25 00:00:00"
    testStartTime ="2022-02-25 00:00:00"
    testEndTime ="2022-02-28 00:00:00"
    
##################################################
#### 1-3. Static Varialbe List
LearningModeList=["train", "test"]
cleanParamList =["Clean", "NoClean"]
##################################################

# 2. Make Save Dataset And Metadata

In [ ]:
for learningMode in LearningModeList:
    for cleanParam in cleanParamList:
        processParam = p1.get_process_param(cleanParam) 
        if learningMode == 'train':
            startTime = trainStartTime
            endTime = trainEndTime

        elif learningMode =='test':
            startTime = testStartTime
            endTime = testEndTime
        
        dbName ='intData_'+datasetName
        dataName =learningMode+cleanParam+'_'+datasetName
        # dataName = getNewDataName()
        data = p1.getData(pathSetting.db_client, dataInfo, integration_freq_sec, processParam, startTime, endTime)
        if DataSaveMode == 'influx':
            p1.save_influx_data(dbName, dataName, data, pathSetting.db_client)
        elif DataSaveMode == 'CSV':
            fileName = p1.save_csv_data(pathSetting.dataFolderPath, dataName, data)
        data[['NH3ppm']].plot()
        plt.show()
            
        p1.saveJsonMeta(pathSetting.DataMetaPath, dataName, processParam, dataInfo,integration_freq_sec, startTime, endTime, cleanParam, DataSaveMode)